In [1]:
!pip install cdsapi numpy urllib3  pandas datetime xarray[complete] 

^C


In [2]:
import xarray as xr

import numpy as np

from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()

import json
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
import zipfile
import os

from zipfile import ZipFile 


In [3]:
from datetime import date,timedelta
time1=(date.today()- timedelta(days = 1)).strftime("%Y-%m-%d")
time1


'2025-02-16'

In [ ]:
import cdsapi
URL = 'https://ads.atmosphere.copernicus.eu/api'
KEY = '87bb8d36-2458-48c2-b5e3-f1fd352291bc'
dataset = "cams-global-atmospheric-composition-forecasts"
lead= [str(i) for i in range(0,121)]

request = {
    "date": [f"{time1}/{time1}"],
    "time": ["12:00"],
    "leadtime_hour": lead,
    "type": ["forecast"],
    "data_format": "netcdf_zip",
    "variable": [
         "2m_dewpoint_temperature",
        
        "dust_aerosol_optical_depth_550nm",
        
        "nitrate_aerosol_optical_depth_550nm",
        "particulate_matter_2.5um",
        "particulate_matter_10um",
        "sea_salt_aerosol_optical_depth_550nm",
        "sulphate_aerosol_optical_depth_550nm",
        "total_aerosol_optical_depth_550nm",
        "convective_precipitation",
        "high_cloud_cover",
        "low_cloud_cover",
        "medium_cloud_cover",
        "total_cloud_cover",
        "most_unstable_convective_available_potential_energy",
        "most_unstable_convective_inhibition"

    ],
    "area": [65, 40, 0, 100]
}

client = cdsapi.Client(url=URL, key=KEY)
client.retrieve(dataset, request).download('test.zip')


2025-02-17 11:17:55,223 INFO [2024-09-26T00:00:00] Watch our [Forum]( https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-17 11:17:56,398 INFO Request ID is 2bef495c-d099-42d5-80bb-2390535cf16a
2025-02-17 11:17:56,587 INFO status has been updated to accepted
2025-02-17 11:18:02,088 INFO status has been updated to running
2025-02-17 11:24:17,842 INFO status has been updated to successful


8821e54dd56bf6f1324178f23c65686a.zip:   0%|          | 0.00/101M [00:00<?, ?B/s]

'test.zip'

In [5]:

with ZipFile("test.zip", 'r') as zObject: 
  
    # Extracting specific file in the zip 
    # into a specific location. 
    zObject.extractall( 
         path=".") 
zObject.close() 

ds=xr.open_dataset('data_sfc.nc')

In [6]:
#ds.valid_time.values

In [ ]:
aod_du=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['duaod550']
aod_ss=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['ssaod550']
aod_su=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['suaod550']
aod=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['aod550']
aod_ni=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['niaod550']
time=ds.valid_time

# rf1 = ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tp'][:, :, :]  # Select a specific time slice
# p =ds.sel(longitude=slice(60,100),latitude=slice(40,0))['msl'][:, :, :]/100  # Select a specific time slice
# tcw=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tcwv']
pm25=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['pm2p5']
pm10=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['pm10']
# t2=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['t2m']

lcc=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['lcc']
mcc=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['mcc']
hcc=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['hcc']
tcc=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['tcc']

cape=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['mucape']
cin=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['mucin']

d2m=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['d2m']
cp=ds.sel(longitude=slice(60,100),latitude=slice(40,0))['cp']


# w10=(u10**2+v10**2)**0.5
# from metpy.units import units
# rf2=rf* units('m').to(units.mm)
rf=rf1.diff(dim='forecast_period').dropna(dim='forecast_period')

In [15]:

def dict_l(data,fm):
  data_dict=data.squeeze().to_dict(data='list', encoding=False)
  def convert_types(obj):
    if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
        return obj.isoformat()  # Convert datetime objects to ISO format
    elif isinstance(obj, timedelta):
        return obj.total_seconds()  # Convert timedelta to total seconds
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert numpy floats to native floats
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert numpy integers to native ints
    raise TypeError(f"Type {type(obj)} not serializable")

  file=f"{fm}.json"
  # Save the dictionary as a JSON file with the extended convert_types function
  with open(file, 'w') as json_file:
      json.dump(data_dict, json_file, indent=4, default=convert_types)
  print('done',file)

In [14]:
time

<xarray.DataArray 'valid_time' (forecast_reference_time: 1, forecast_period: 121)> Size: 968B
[121 values with dtype=datetime64[ns]]
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 968B 00:00:00 ...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 8B 2025...
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] 968B ...
Attributes:
    standard_name:  time
    long_name:      time

In [ ]:
 dict_l((t2.round(2)-273.15).round(2),"2m_temp")
 dict_l((rf*1000).round(2),"rf")
 dict_l(p,"msl")
 dict_l(tcw.round(2),"tcwv")
 dict_l(1e9*pm25.round(2),"PM25")
 dict_l(1e9*pm10.round(2),"PM10")

 dict_l(lcc.round(2),"LCC")
 dict_l(mcc.round(2),"MCC")
 dict_l(hcc.round(2),"HCC")
 dict_l(tcc.round(2),"TCC")

 dict_l(cape.round(2),"CAPE")
 dict_l(cin.round(2),"CIN")
 dict_l((d2m.round(2))-273.15,"D2M")
 dict_l((cp.round(2)*1000),"CP")

 dict_l(aod_ss.round(2),"AOD_SEA")
 dict_l(aod_su.round(2),"AOD_SUL")
 dict_l(aod_ni.round(2),"AOD_NIT")
 dict_l(aod_du.round(2),"AOD_DUST")
 dict_l(aod.round(2),"AOD_TOT")

# dict_l(time,"TIME")






done 2m_temp.json
done rf.json
done msl.json
done tcwv.json
done PM25.json
done PM10.json
done LCC.json
done MCC.json
done HCC.json
done TCC.json
done CAPE.json
done CIN.json
done D2M.json
done CP.json
done AOD_SEA.json
done AOD_SUL.json
done AOD_NIT.json
done AOD_DUST.json
done AOD_TOT.json


array(['2025-02-16T22:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
# import numpy as np

# # Define the colors in RGBA format
# colors = {
#     "blue": (0, 0, 255, 1),
#     "aqua": (0, 255, 255, 1),
#     "green": (0, 255, 0, 1),
#     "yellow": (255, 255, 0, 1),
#     "orange": (255, 165, 0, 1),
#     "red": (255, 0, 0, 1)
# }

# # Define color stops and corresponding positions
# color_stops = [
#     (0.0, (0, 0, 0, 1)),  # Changed to tuple
#     (0.12, (0, 0, 255, 1)),  # Changed to tuple
#     (0.25, (0, 255, 255, 1)),  # Changed to tuple
#     (0.38, (0, 255, 0, 1)),  # Changed to tuple
#     (0.5, (255, 255, 0, 1)),  # Changed to tuple
#     (0.62, (255, 165, 0, 1)),  # Changed to tuple
#     (0.75, (255, 0, 0, 1)),  # Changed to tuple
#     (0.88, (238, 130, 238, 1)),  # Changed to tuple
#     (1.0, (255, 255, 255, 1))  # Changed to tuple
# ]

# # Generate 100 interpolated values
# n_points = 200
# interpolated_colors = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest stops
#     for j in range(len(color_stops) - 1):
#         if color_stops[j][0] <= pos <= color_stops[j + 1][0]:
#             # Interpolate between the two stops
#             start_pos, start_color = color_stops[j]
#             end_pos, end_color = color_stops[j + 1]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             interpolated_colors.append([round(pos, 2), rgba_string])
#             break

# interpolated_colors


In [ ]:
# # Number of interpolated colors
# n_points = 256

# # Prepare an empty list for the AMWG256 colormap
# amwg256 = []

# # Define the positions of the input colors in the sequence
# color_positions = [i / (len(color_sequence) - 1) for i in range(len(color_sequence))]

# # Generate interpolated colors for 256 points
# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(color_positions) - 1):
#         if color_positions[j] <= pos <= color_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = color_positions[j], color_positions[j + 1]
#             start_color, end_color = color_map[color_sequence[j]], color_map[color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             amwg256.append([round(pos, 3), rgba_string])
#             break

# amwg256  # Display the first 10 color stops to verify


In [ ]:
# # Define the colormap structure for GMT_panoply with stops: Black, Blue, Aqua, Green, Yellow, Orange, Red, White
# gmt_color_sequence = ["Bk", "Bl", "Aq", "Gr", "Ye", "Or", "Re", "Wh"]

# # Define the positions of the input colors
# gmt_positions = [i / (len(gmt_color_sequence) - 1) for i in range(len(gmt_color_sequence))]

# # Interpolating 256 colors for the GMT_panoply colormap
# gmt_panoply_256 = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(gmt_positions) - 1):
#         if gmt_positions[j] <= pos <= gmt_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = gmt_positions[j], gmt_positions[j + 1]
#             start_color, end_color = color_map[gmt_color_sequence[j]], color_map[gmt_color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             gmt_panoply_256.append([round(pos, 3), rgba_string])
#             break

# gmt_panoply_256  # Display the first 10 color stops to verify
